In [149]:
import pandas as pd

# 读取wording.xlsx文件，第一个sheet保存为df1，第二个sheet保存为df2，忽略列名
df1 = pd.read_excel('wording.xlsx', sheet_name=0, header=None)
df2 = pd.read_excel('wording.xlsx', sheet_name=1, header=None)

# 对df1所有列合并为一列，列名为sheet1_term
df1 = pd.concat([df1[col] for col in df1.columns], ignore_index=True)
df1.name = 'sheet1_term'

# 全部转小写
df1 = df1.str.lower()

# 按值排序
df1 = df1.sort_values()

# 去空值、去nan值、去重复值
df1 = df1.dropna().drop_duplicates()

# 重置索引
df1 = df1.reset_index(drop=True)

# 转为dataframe类型
df1 = pd.DataFrame(df1)

# 对df2进行相同的操作
df2 = pd.concat([df2[col] for col in df2.columns], ignore_index=True)
df2.name = 'sheet2_term'
df2 = df2.str.lower()
df2 = df2.sort_values()
df2 = df2.dropna().drop_duplicates()
df2 = df2.reset_index(drop=True)
df2 = pd.DataFrame(df2)
 


import os

# 获取当前工作目录
current_dir = os.getcwd()

# 拼接orgdata文件夹路径
orgdata_dir = os.path.join(current_dir, 'orgdata')

# 遍历orgdata文件夹中所有文件
pdflist = []
for file in os.listdir(orgdata_dir):
    # 判断文件是否为pdf文件
    if file.endswith('.pdf'):
        # 将文件名加上前缀"orgdata/"并保存到pdflist数组中
        pdflist.append(os.path.join('orgdata', file))




# 取出df1的第一个元素变量名是 term
term = df1.iloc[0][0]

# 取出pdflist的第一个元素为变量名是 pdf
pdf = pdflist[0]




import fitz
def pdf_count(pdf):

    # 用PyMuPDF 库读取变量 pdf
    doc = fitz.open(pdf)

    # 读取文件将所有文本拼到一起,保存为变量 alltext
    alltext = ""
    for page in doc:
        alltext += page.get_text()

    # 对 alltext 用空格进行分割得到数组 textlist
    textlist = alltext.split()

    # textlist 所有元素字符字母转小写,得到数组 lowtextlist
    lowtextlist = [word.lower() for word in textlist]



    def getcount(term, lowtextlist):
        # 初始化变量 num 为0, 令num对 变量term 在 lowtextlist 中出现的次数
        num = lowtextlist.count(term)
        
        # 初始化变量 fb10_num 为0
        fb10_num = 0
        
        # 初始化变量 fbtext 为 none
        fbtext = None
        
        # 如果 num 是大于0的, 则取出 term 所在的索引位置,左边界-10, 右边界+10 切片数组, 保存为 fbtext
        if num > 0:
            index = lowtextlist.index(term)
            fbtext = lowtextlist[index-10:index+10]
        
        # 当 变量 fbtext 不是 none 时, 遍历全局变量 df2['trem'] 的所有元素, 将它们出现在 fbtext 的总次数保存到变量 fb10_num 中
        if fbtext is not None:
            for t in df2['sheet2_term']:
                fb10_num += fbtext.count(t)
        
        # 返回值为 num 与 fbtext ,fb10_num
        return num, fbtext, fb10_num

    # 对df1的term列使用apply方法执行getcount函数，将第一个返回值保存到'# of sheet1_term'列中，第二个返回值保存到'sheet1_termfb10'列中，第三个返回值保存到'# of sheet2_term_in_sheet1_termfb10'列中
    df1[['# of sheet1_term', 'sheet1_termfb10', '# of sheet2_term_in_sheet1_termfb10']] = df1['sheet1_term'].apply(lambda x: pd.Series(getcount(x, lowtextlist)))

    # 对df2的term列使用apply方法执行getcount函数，将第一个返回值保存到'# of sheet2_term'列中
    df2['# of sheet2_term'] = df2['sheet2_term'].apply(lambda x: getcount(x, lowtextlist)[0])

    # 创建空字典 resdic
    resdic = {}

    # 将 pdf 文件名添加到 resdic 中，键名为 'file'，键值为 pdf 文件名
    resdic['file'] = pdf

    # 将 df1 '# of sheet1_term' 列的求和值添加到 resdic 中，键名为 'total # of sheet1_term'
    resdic['total # of sheet1_term'] = df1['# of sheet1_term'].sum()

    # 将 df1 '# of sheet2_term_in_sheet1_termfb10' 列的求和值添加到 resdic 中，键名为 'total # of sheet2_term_in_sheet1_termfb10'
    resdic['total # of sheet2_term_in_sheet1_termfb10'] = df1['# of sheet2_term_in_sheet1_termfb10'].sum()

    # 将 df2 '# of sheet2_term' 列的求和值添加到 resdic 中，键名为 'total # of sheet2_term'
    resdic['total # of sheet2_term'] = df2['# of sheet2_term'].sum()

    # 将 lowtextlist 的长度添加到 resdic 中，键名为 'total # of pdf'
    resdic['total # of pdf'] = len(lowtextlist)

    # 返回 resdic

  
    # 创建变量 filen
    filen = 'output.xlsx'

    # 将 df1,与df2 以 sheet1 不有sheet2的不同页面 保存到同一个excel表格中
    with pd.ExcelWriter(filen) as writer:
        df1.to_excel(writer, sheet_name='sheet1', index=False)
        df2[~df2['sheet2_term'].isin(df1['sheet1_term'])].to_excel(writer, sheet_name='sheet2', index=False)# 将 变量 pdf 的 ".pdf" 修改为 ".xlsx",保存到 变量filen中
    filen = pdf.replace('.pdf', '.xlsx')

    # 将 df1,与df2 以 sheet1 不有sheet2的不同页面 保存到同一个 命为 变量 filen 的excel表格中
    with pd.ExcelWriter(filen) as writer:
        df1.to_excel(writer, sheet_name='sheet1', index=False)
        df2[~df2['sheet2_term'].isin(df1['sheet1_term'])].to_excel(writer, sheet_name='sheet2', index=False)

# 将 resdic 转换为 DataFrame，列名为字典的 key
    df_resdic = pd.DataFrame.from_dict(resdic, orient='index').T

    # 返回 df_resdic
    return df_resdic




# 创建空列表 alldf
alldf = []

# 遍历 pdflist 中的每个pdf文件，执行pdf_count方法，并将返回值添加到 alldf 列表中
for pdf in pdflist:
    alldf.append(pdf_count(pdf))

# 使用 pandas concat 方法将 alldf 列表中的所有数据框连接起来
result = pd.concat(alldf)


result



,file,total # of sheet1_term,total # of sheet2_term_in_sheet1_termfb10,total # of sheet2_term,total # of pdf
0,orgdata\100080.pdf,10.0,5.0,16,906
0,orgdata\100135.pdf,249.0,15.0,249,14153
0,orgdata\10016.pdf,9.0,9.0,29,694
0,orgdata\100338.pdf,20.0,9.0,25,887
0,orgdata\100401.pdf,70.0,9.0,32,4645
...,...,...,...,...,...
0,orgdata\9135.pdf,19.0,10.0,42,1330
0,orgdata\9203.pdf,77.0,11.0,45,10531
0,orgdata\9555.pdf,19.0,6.0,23,1239
0,orgdata\9774.pdf,5.0,1.0,9,862


In [150]:
result.to_excel("all.xlsx")